In [11]:
import csv
import os
import sys
from collections import defaultdict

def convert_csv_format(input_file, output_file):
    """
    Converts a detector configuration file from the old format to the new format.
    
    Args:
        input_file: Path to the input CSV file (old format)
        output_file: Path to the output CSV file (new format)
    """
    # Initialize data structure to store the converted data
    converted_data = []
    
    # Read the input CSV file
    try:
        with open(input_file, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            data = list(reader)
    except Exception as e:
        print(f"Error reading {input_file}: {e}")
        return False
    
    # Get current date as 1/1/2020 (default)
    current_date = "1/1/2020"
    
    # Add header with date
    converted_data.append(["", "", current_date])
    
    # Process traffic movement (TM) sections
    movement_mapping = {
        'EBL': 'EBL', 'EBT': 'EBT', 'EBR': 'EBR',
        'WBL': 'WBL', 'WBT': 'WBT', 'WBR': 'WBR',
        'NBL': 'NBL', 'NBT': 'NBT', 'NBR': 'NBR',
        'SBL': 'SBL', 'SBT': 'SBT', 'SBR': 'SBR'
    }
    
    # Map row indices to their movement types
    row_to_movement = {}
    for i, row in enumerate(data):
        if row and row[0] in movement_mapping:
            row_to_movement[i] = row[0]
    
    # Process movement data
    for movement_type in ['EBL', 'EBT', 'EBR', 'WBL', 'WBT', 'WBR', 'NBL', 'NBT', 'NBR', 'SBL', 'SBT', 'SBR']:
        # Find rows with this movement type
        detectors = []
        for i, row in enumerate(data):
            if i in row_to_movement and row_to_movement[i] == movement_type:
                # Add valid detector numbers from this row
                detectors.extend([cell for cell in row[1:] if cell and cell.strip()])
        
        # Add to converted data
        if detectors:
            # Join multiple detectors with commas and quote if there are multiple
            detector_str = ','.join(detectors)
            converted_data.append(["TM:", movement_type, detector_str])
        else:
            # If no detectors found, still include the movement type with empty value
            converted_data.append(["TM:", movement_type, ""])
    
    # Process Exclude section
    exclude_types = ["Detector", "Phase", "Status"]
    exclude_data = {}
    
    # Find exclude section data
    for exclude_type in exclude_types:
        exclude_data[exclude_type] = []
        for row in data:
            if row and row[0] == exclude_type:
                # Get all non-empty values
                values = [cell for cell in row[1:] if cell and cell.strip()]
                if values:
                    exclude_data[exclude_type] = values
    
    # Add exclude entries to converted data
    for exclude_type in exclude_types:
        if exclude_type in exclude_data and exclude_data[exclude_type]:
            # Special handling for Status values with commas
            if exclude_type == "Status":
                # Check if any values have commas inside them
                processed_values = []
                for val in exclude_data[exclude_type]:
                    if ',' in val:
                        # Format as [Y,R] instead of "Y,R"
                        processed_val = val.replace('"', '')  # Remove existing quotes
                        if not processed_val.startswith('['):
                            processed_val = f"[{processed_val}]"  # Add brackets
                        processed_values.append(processed_val)
                    else:
                        processed_values.append(val)
                values_str = ','.join(processed_values)
            else:
                values_str = ','.join(exclude_data[exclude_type])
            
            converted_data.append(["Exc:", exclude_type, values_str])
        else:
            converted_data.append(["Exc:", exclude_type, ""])
    
    # Process Arrivals and Occupancy sections
    arrival_types = ["P2 Arrival", "P4 Arrival", "P6 Arrival", "P8 Arrival"]
    occupancy_types = [
        "P1 Occupancy", "P2 Occupancy", "P3 Occupancy", "P4 Occupancy",
        "P5 Occupancy", "P6 Occupancy", "P7 Occupancy", "P8 Occupancy"
    ]
    
    # Find and process arrivals
    for arrival_type in arrival_types:
        detectors = []
        for row in data:
            if row and row[0] == arrival_type:
                # Add valid detector numbers
                detectors.extend([cell for cell in row[1:] if cell and cell.strip()])
        
        # Add to converted data
        if detectors:
            detector_str = ','.join(detectors)
            converted_data.append(["Plt:", arrival_type, detector_str])
        else:
            converted_data.append(["Plt:", arrival_type, ""])
    
    # Find and process occupancy
    for occupancy_type in occupancy_types:
        detectors = []
        for row in data:
            if row and row[0] == occupancy_type:
                # Add valid detector numbers
                detectors.extend([cell for cell in row[1:] if cell and cell.strip()])
        
        # Add to converted data
        if detectors:
            detector_str = ','.join(detectors)
            converted_data.append(["Plt:", occupancy_type, detector_str])
        else:
            converted_data.append(["Plt:", occupancy_type, ""])
    
    # Add stop bar entries (not in old format, but required in new format)
    for phase in range(1, 9):
        converted_data.append(["Plt:", f"P{phase} Stop Bar", ""])
    
    # Add default R1, R2, and B entries - without extra quotes
    converted_data.append(["RB:", "R1", "1,2|3,4"])
    converted_data.append(["RB:", "R2", "5,6|7,8"])
    converted_data.append(["RB:", "B", ""])
    
    # Write to the output CSV file with specific quoting behavior
    try:
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
            writer.writerows(converted_data)
        return True
    except Exception as e:
        print(f"Error writing to {output_file}: {e}")
        return False

def process_all_intersections(spm_dir):
    """
    Process all intersections in the SPM directory structure.
    
    Args:
        spm_dir: Path to the SPM directory containing Intersections folder
    """
    intersections_dir = os.path.join(spm_dir, "Intersections")
    
    if not os.path.exists(intersections_dir):
        print(f"Intersections directory not found at {intersections_dir}")
        return
    
    # Track statistics
    total_intersections = 0
    skipped_intersections = 0
    converted_intersections = 0
    failed_intersections = 0
    
    # Get list of all intersection directories
    intersection_dirs = [d for d in os.listdir(intersections_dir) 
                         if os.path.isdir(os.path.join(intersections_dir, d))]
    
    print(f"Found {len(intersection_dirs)} intersection directories")
    
    # Process each intersection
    for intersection in intersection_dirs:
        total_intersections += 1
        intersection_path = os.path.join(intersections_dir, intersection)
        config_dir = os.path.join(intersection_path, "configuration")
        
        # Skip if configuration directory doesn't exist
        if not os.path.exists(config_dir):
            print(f"Skipping {intersection}: No configuration directory")
            skipped_intersections += 1
            continue
        
        # Define input and output file paths
        input_file = os.path.join(config_dir, "dets.csv")
        output_file = os.path.join(config_dir, "int_cfg.csv")
        
        # Skip if input file doesn't exist
        if not os.path.exists(input_file):
            print(f"Skipping {intersection}: No dets.csv file")
            skipped_intersections += 1
            continue
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {intersection}: int_cfg.csv already exists")
            skipped_intersections += 1
            continue
        
        # Convert the file
        print(f"Converting {intersection}...")
        success = convert_csv_format(input_file, output_file)
        
        if success:
            print(f"Successfully converted {intersection}")
            converted_intersections += 1
        else:
            print(f"Failed to convert {intersection}")
            failed_intersections += 1
    
    # Print summary
    print("\nConversion Summary:")
    print(f"Total intersections found: {total_intersections}")
    print(f"Intersections skipped (already converted or missing files): {skipped_intersections}")
    print(f"Intersections successfully converted: {converted_intersections}")
    print(f"Intersections failed to convert: {failed_intersections}")

In [6]:
import os
import sys
import time
from datetime import datetime, timedelta

def delete_recent_int_cfg_files(spm_dir, hours=1):
    """
    Walks through the SPM directory structure and deletes any int_cfg.csv files
    that were created within the specified number of hours.
    
    Args:
        spm_dir: Path to the SPM directory containing Intersections folder
        hours: Number of hours to look back (default: 1)
    """
    intersections_dir = os.path.join(spm_dir, "Intersections")
    
    if not os.path.exists(intersections_dir):
        print(f"Intersections directory not found at {intersections_dir}")
        return
    
    # Calculate the cutoff time (current time minus specified hours)
    cutoff_time = time.time() - (hours * 3600)
    
    # Track statistics
    total_intersections = 0
    files_deleted = 0
    files_skipped = 0
    
    # Get list of all intersection directories
    intersection_dirs = [d for d in os.listdir(intersections_dir) 
                        if os.path.isdir(os.path.join(intersections_dir, d))]
    
    print(f"Found {len(intersection_dirs)} intersection directories")
    print(f"Searching for int_cfg.csv files created in the past {hours} hour(s)...")
    
    # Process each intersection
    for intersection in intersection_dirs:
        total_intersections += 1
        intersection_path = os.path.join(intersections_dir, intersection)
        config_dir = os.path.join(intersection_path, "configuration")
        
        # Skip if configuration directory doesn't exist
        if not os.path.exists(config_dir):
            continue
        
        # Check for int_cfg.csv file
        int_cfg_path = os.path.join(config_dir, "int_cfg.csv")
        
        if not os.path.exists(int_cfg_path):
            continue
        
        # Check file creation time
        file_creation_time = os.path.getmtime(int_cfg_path)
        
        if file_creation_time > cutoff_time:
            # File was created within the specified time period
            creation_time_str = datetime.fromtimestamp(file_creation_time).strftime('%Y-%m-%d %H:%M:%S')
            print(f"Deleting {intersection}/configuration/int_cfg.csv (created at {creation_time_str})")
            
            try:
                os.remove(int_cfg_path)
                files_deleted += 1
                print(f"Successfully deleted {int_cfg_path}")
            except Exception as e:
                print(f"Error deleting {int_cfg_path}: {e}")
        else:
            files_skipped += 1
    
    # Print summary
    print("\nDeletion Summary:")
    print(f"Total intersections examined: {total_intersections}")
    print(f"int_cfg.csv files deleted (created in the past {hours} hour(s)): {files_deleted}")
    print(f"int_cfg.csv files skipped (older than {hours} hour(s)): {files_skipped}")


In [10]:
delete_recent_int_cfg_files('./')

Found 42 intersection directories
Searching for int_cfg.csv files created in the past 1 hour(s)...
Deleting 2ND & AMERICAN LEGION/configuration/int_cfg.csv (created at 2025-02-28 12:17:33)
Successfully deleted ./Intersections\2ND & AMERICAN LEGION\configuration\int_cfg.csv
Deleting 3RD & AIRBASE/configuration/int_cfg.csv (created at 2025-02-28 12:18:43)
Successfully deleted ./Intersections\3RD & AIRBASE\configuration\int_cfg.csv
Deleting 3RD & AMERICAN LEGION/configuration/int_cfg.csv (created at 2025-02-28 12:17:33)
Successfully deleted ./Intersections\3RD & AMERICAN LEGION\configuration\int_cfg.csv
Deleting 5TH & AIRBASE/configuration/int_cfg.csv (created at 2025-02-28 12:25:06)
Successfully deleted ./Intersections\5TH & AIRBASE\configuration\int_cfg.csv
Deleting ACHD_Chinden-SH16/configuration/int_cfg.csv (created at 2025-02-28 12:17:48)
Successfully deleted ./Intersections\ACHD_Chinden-SH16\configuration\int_cfg.csv
Deleting ACHD_Eagle-Chinden/configuration/int_cfg.csv (created at 

In [ ]:

if __name__ == "__main__":
    # Check if SPM directory and hours are provided as command line arguments
    if len(sys.argv) > 2:
        spm_dir = sys.argv[1]
        hours = float(sys.argv[2])
    elif len(sys.argv) > 1:
        spm_dir = sys.argv[1]
        hours = 1  # Default to 1 hour
    else:
        # Use current directory as default
        spm_dir = os.getcwd()
        hours = 1  # Default to 1 hour
    
    print(f"Starting deletion of recent int_cfg.csv files from SPM directory: {spm_dir}")
    print(f"Will delete files created in the past {hours} hour(s)")
    
    # Ask for confirmation
    confirmation = input("Are you sure you want to proceed? (y/n): ")
    if confirmation.lower() == 'y':
        delete_recent_int_cfg_files(spm_dir, hours)
    else:
        print("Operation cancelled.")

In [13]:
process_all_intersections('./')

Found 42 intersection directories
Skipping 2ND & AMERICAN LEGION: int_cfg.csv already exists
Skipping 3RD & AIRBASE: int_cfg.csv already exists
Skipping 3RD & AMERICAN LEGION: int_cfg.csv already exists
Skipping 5TH & AIRBASE: int_cfg.csv already exists
Skipping ACHD_Chinden-SH16: int_cfg.csv already exists
Skipping ACHD_Eagle-Chinden: int_cfg.csv already exists
Skipping ACHD_Eagle-Colchester: int_cfg.csv already exists
Skipping ACHD_Eagle-Fairview: int_cfg.csv already exists
Skipping ACHD_Eagle-Franklin: int_cfg.csv already exists
Skipping ACHD_Eagle-HobbleCreek: int_cfg.csv already exists
Skipping ACHD_Eagle-I84EB: int_cfg.csv already exists
Skipping ACHD_Eagle-I84WB: int_cfg.csv already exists
Skipping ACHD_Eagle-IslandWoods: int_cfg.csv already exists
Skipping ACHD_Eagle-McMillan: int_cfg.csv already exists
Skipping ACHD_Eagle-Pine: int_cfg.csv already exists
Skipping ACHD_Eagle-Riverside: int_cfg.csv already exists
Skipping ACHD_Eagle-RiverValley: int_cfg.csv already exists
Skippi

In [ ]:

if __name__ == "__main__":
    # Check if SPM directory is provided as command line argument
    if len(sys.argv) > 1:
        spm_dir = sys.argv[1]
    else:
        # Use current directory as default
        spm_dir = os.getcwd()
    
    print(f"Starting batch conversion from SPM directory: {spm_dir}")
    process_all_intersections(spm_dir)